In [ ]:
!pip install sentencepiece
!pip install gluonnlp|
!pip install mxnet
!pip install tensorflow_addons
!pip install emoji
!pip install soynlp
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.9 MB/s eta 0:00:00
/bin/bash: -c: line 2: syntax error: unexpected end of file
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 21.5 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.4 MB/s eta 0:00:00


In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00


In [ ]:
import tensorflow_addons
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
import os
import unicodedata
import utils
import time
import torch
import constants
import re
import emoji


from transformers import PreTrainedTokenizer, TFBertModel, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding,logging, AdamW
from tensorflow_addons.optimizers import RectifiedAdam
from keras.models import load_model
from shutil import copyfile
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from datasets import load_dataset
from utils import clean, make_current_datetime_dir, compute_metrics, preprocess_data,multi_label_metrics
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR
from soynlp.normalizer import repeat_normalize

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


ModuleNotFoundError: ignored

### Google Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Step 1. 감성분석 모델

In [ ]:
path = '/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/감성분석/Result/8 22 결과'
model_a = AutoModelForSequenceClassification.from_pretrained(path)
tokenizer_a = AutoTokenizer.from_pretrained(path)
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
model_a = model_a.to(device)  # 모델을 디바이스로 이동

In [ ]:
def sentimental_predict(sent):
    # 모델을 평가 모드로 설정
    model_a.eval()

    # 문장을 토큰화
    tokenized_sent = tokenizer_a(
        sent,
        return_tensors='pt',
        truncation=True,
        add_special_tokens=True,
        max_length=128
    )

    # 토큰화된 문장을 디바이스로 이동
    tokenized_sent = {k: v.to(device) for k, v in tokenized_sent.items()}

    # 모델의 예측 실행
    with torch.no_grad():
        outputs = model_a(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
        )

    # 로짓 추출 및 CPU로 이동
    logits = outputs[0].detach().cpu()

    # 결과 분류
    result = logits.argmax(-1).item()
    if result == 0:
      print('1. 감성분석의 결과, 이 문장은 부정적인 문장입니다.')
    elif result == 1:
      print("1. 감성분석의 결과, 이 문장은 부정적이지 않은 문장입니다.")


## Step 2: 언어감수성 이진분류 모델

In [ ]:
path = '/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/언어감수성 판단 유무/Result/언어감수성 판단 유무'
model_b = AutoModelForSequenceClassification.from_pretrained(path)
tokenizer_b = AutoTokenizer.from_pretrained(path)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
model_b = model_b.to(device)  # 모델을 디바이스로 이동

def predict(sent):
    # 모델을 평가 모드로 설정
    model_b.eval()

    # 문장을 토큰화
    tokenized_sent = tokenizer_b(
        sent,
        return_tensors='pt',
        truncation=True,
        add_special_tokens=True,
        max_length=128
    )

    # 토큰화된 문장을 디바이스로 이동
    tokenized_sent = {k: v.to(device) for k, v in tokenized_sent.items()}

    # 모델의 예측 실행
    with torch.no_grad():
        outputs = model_b(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
        )

    # 로짓 추출 및 CPU로 이동
    logits = outputs[0].detach().cpu()

    # 결과 분류
    result = logits.argmax(-1).item()
    if result == 0:
      print('2. 언어감수성 분석 결과, 이 문장은 언어감수성이 높습니다')
    elif result == 1:
      print("2. 언어감수성 분석 결과, 이 문장은 언어감수성이 낮습니다.")



In [ ]:
predict("오늘 반팔티 입었습니다")

언어감수성이 낮습니다.


## Step 3: 언어감수성 카테고리 분류 모델

In [ ]:
ckpt_path = '/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/언어감수성 카테고리 판단 유무/Result/checkpoint-131560'
model_c = AutoModelForSequenceClassification.from_pretrained(ckpt_path).to(device)
tokenizer_c = AutoTokenizer.from_pretrained(ckpt_path)

In [ ]:
def infer(sentences):
    global model_c
    global tokenizer_c
    global device
    id2label = constants.ID2LABEL_KOR
    results = []

    for sentence in sentences:
        sentence = clean(sentence)

        infer_stime = time.time()
        encoding = tokenizer_c(sentence, return_tensors='pt').to(device)
        outputs = model_c(**encoding)
        logits = outputs.logits
        sigmoid = torch.nn.Sigmoid()
        preds = sigmoid(logits.squeeze())
        infer_etime = time.time()

        result = {'문장': sentence,
                  '추론시간': infer_etime - infer_stime
                  }

        for id, label in id2label.items():
            prob = preds[id].item()
            result[label] = prob

        results.append(result)

    results = pd.DataFrame(results)
    category = results.columns[np.argmax(results.iloc[:, 1:])+1]

    return category

In [ ]:
sentence = '나는 오늘 너무 행복해'
sentences = [sentence]

model_c.eval()
ret = infer(sentences)
display(ret)



'언어감수성_O'

## Step 4: 사전에 구축한 대안어 사전 로드

In [ ]:
!pip install konlpy
import pandas as pd
import time

from konlpy.tag import Kkma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 41.7 MB/s eta 0:00:00


In [ ]:
dictionary = pd.read_excel("/content/drive/MyDrive/데이터 청년 캠퍼스/사전/5조 챗지피티 애드온 대안어 사전.xlsx",sheet_name=None)
df1 = list(dictionary.values())[0]
df1 = df1.drop("Unnamed: 0",axis=1)
df2 = list(dictionary.values())[1]
df2 = df2[['from','to']]
df3 = list(dictionary.values())[2]
df3 = df3.drop("Unnamed: 0",axis=1)
df4 = list(dictionary.values())[3]
df4 = df4.drop("Unnamed: 0",axis=1)
df5 = list(dictionary.values())[4]
df5 = df5.drop("Unnamed: 0",axis=1)
df6 = list(dictionary.values())[5]
df6 = df6.drop('ㄱ',axis=1)
df6.columns = ['from','to']
dfa = pd.concat([df1,df2,df3,df4,df5,df6],axis = 0)
dfa.index = range(len(dfa))

diction = {i:j for i,j in zip(dfa['from'], dfa['to'])}


for i,j in diction.items():
  try:
    if np.isnan(j):
      diction[i] = 'No'
  except:
    pass

## Step 5. 대안어 사전 출력 로직

In [ ]:
kkma = Kkma()
verb = ['VXV','VCP','VCN','MAG','MAC','IC','JKS','JKC','JKG','JKO','JKM','JKI','JKQ','JX','JC','EPH','EPT','EPP','EFN','EFQ','EFO','EFA','EFI','EFR','ECE','ECD','ECS','ETD','SF','SP','SS','SE','SO','SW','OL','OH','ON']

In [ ]:
def Recsys_Sentiment_word_Version_kkma(sentence):
  word = set()
  pos_sent =kkma.pos(sentence)
  verb = ['VXV','VCP','VCN','MAG','MAC','IC','JKS','JKC','JKG','JKO','JKM','JKI','JKQ','JX','JC','EPH','EPT','EPP','EFN','EFQ','EFO','EFA','EFI','EFR','ECE','ECD','ECS','ETD','SF','SP','SS','SE','SO','SW','OL','OH','ON']

  ### 1단계 로직 짜기.
  First = [i for i in pos_sent if i[1] not in verb]
  for word_token in First:
    if word_token[0] in list(dfa['from']):
      word.add(word_token[0])

  ### 2단계 로직 짜기
  import nltk

  for i in list(nltk.bigrams(First)):
    if i[0][0] + i[1][0] in list(dfa['from']):
      word.add(i[0][0] + i[1][0])

  ### 3단계 로직 짜기
  import re

  sent_split = re.sub(r'[.,!?(){}[\]<>%/@;:"^#*&…~\-]', '', sentence).split()
  for i in pos_sent:
    if i[0] in sent_split and i[1] in verb:
      sent_split.remove(i[0])
  for word_token in sent_split:
    if word_token in list(dfa['from']):
      word.add(word_token)


  ### 마무리 코드 치기
  number = 0

  for i in word:
    if diction[i] != 'No' :
      print("단어 \"{}\" 은/는 언어감수성이 낮은 단어로 \"{}\"로 표현하는 것이 좋아보입니다.".format(i,diction[i]))
      number = 1
    else:
      print("단어 \"{}\" 은/는 언어감수성이 낮은 단어표현입니다.".format(i,diction[i]))
      number = 1
  if number == 0:
    print("위 문장에는 언어감수성이 낮은 단어가 없습니다.")

In [ ]:
Recsys_Sentiment_word_Version_kkma('나는 오늘 아내랑 같이 밥을 먹으러 갈거야')

단어 "아내" 은/는 언어감수성이 낮은 단어로 "배우자"로 표현하는 것이 좋아보입니다.


## Step 6: 사전 자동화 구축 (미완)

In [ ]:
def Autodict(sentence):
  result = []
  noun_sent = kkma.noun(sentence)
  for i in noun_sent:
    if predict(i) == '언어감수성이 낮습니다':
      result.append(i)

  return result

### 프로토타입 녹화 부분

In [ ]:
sentence = input()

### 1. 감성분석 모델
start_time = time.time()
sentimental_predict(sentence)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"감성분석 모델 실행 시간: {elapsed_time:.4f}초")
print()

### 2. 언어감수성 유무 판단 모델
start_time = time.time()
predict(sentence)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"언어감수성 이진분류 모델 실행 시간: {elapsed_time:.4f}초")
print()

### 3. 언어감수성 카테고리 판단 모델
start_time = time.time()
ret = infer([sentence])
print("3. 언어감수성 카테고리 모델 결과, 이 문장은 \"{}\" 카테고리에 해당됩니다.".format(ret))
end_time = time.time()
elapsed_time = end_time - start_time
print(f"언어감수성 카테고리 모델 코드 실행 시간: {elapsed_time:.4f}초")
print()

### 4. 추천 단어 판단
print("4. 추천 로직 코드 실행")
start_time = time.time()
Recsys_Sentiment_word_Version_kkma(sentence)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"추천 로직 코드 실행 시간: {elapsed_time:.4f}초")
print()
